In [1]:
import csv
import signal
import ssl
import time
from multiprocessing import Pool, cpu_count, Value, Lock
from os import getpid, kill
import OpenSSL
import requests
from urllib3 import exceptions
import base64
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options

In [2]:
client_id = '9c7dc5330ca142a691fd0959682d0761'
client_secret = '5ff88db8af1d4b1f97828479a33b5c1e'

In [3]:
def get_auth_token():
    tmp = client_id + ':' + client_secret
    tmp = tmp.encode('ascii')
    base64_code = base64.b64encode(tmp)
    base64_code = base64_code.decode('ascii')
    try:
        url = 'https://accounts.spotify.com/api/token'
        request = requests.post(url=url, headers={'Authorization': 'Basic ' + str(base64_code)}, data={'grant_type': 'client_credentials'})
    except OpenSSL.SSL.SysCallError as e:
        print(e)
        return None
    except ssl.SSLError as e:
        print(e)
        return None
    except exceptions.MaxRetryError as e:
        print(e)
        return None
    except requests.exceptions.SSLError as e:
        print(e)
        return None
    except:
        print("Unknown error. Unable to get further information.")
        return None

    data = request.json()
    return data['access_token'], data['expires_in']

In [4]:
def fetch_playlists(authorization):
    try:
        url = 'https://api.spotify.com/v1/users/gangstarappa/playlists'
        request = requests.get(url=url, headers={'Authorization': 'Bearer ' + authorization})
    except OpenSSL.SSL.SysCallError as e:
        print(e)
        return None
    except ssl.SSLError as e:
        print(e)
        return None
    except exceptions.MaxRetryError as e:
        print(e)
        return None
    except requests.exceptions.SSLError as e:
        print(e)
        return None
    except:
        print("Unknown error. Unable to get further information.")
        return None

    data = request.json()
    return data

In [5]:
def fetch_playlist(authorization, playlist_id):
    ret = list()
    try:
        offset = 0
        for i in range(10):
            url = 'https://api.spotify.com/v1/playlists/' + playlist_id + '/tracks'
            request = requests.get(url=url, headers={'Authorization': 'Bearer ' + authorization}, params={'offset': str(offset)})
            while request.status_code != 200:
                if request.status_code == 429:
                    retry = int(request.headers['Retry-After']) + 1
                    print(f"Process {getpid()}: Response code {request.status_code}. Retrying in {retry}s..")
                    time.sleep(retry)
                    request = requests.get(url=url, headers={'Authorization': 'Bearer ' + authorization}, params={'offset': str(offset)})
                elif request.status_code == 403:
                    print(f"Process {getpid()}: Response code {request.status_code}. Killing process.")
                    kill(getpid(), signal.SIGTERM)
                    return None
                else:
                    print(f"Response code {request.status_code}. Ending query chain.")
                    return None

            offset += 100
            data = request.json()
            # print(data)
            ret.append(data)
    except OpenSSL.SSL.SysCallError as e:
        print(e)
        return None
    except ssl.SSLError as e:
        print(e)
        return None
    except exceptions.MaxRetryError as e:
        print(e)
        return None
    except requests.exceptions.SSLError as e:
        print(e)
        return None
    except:
        print("Unknown error. Unable to get further information.")
        return None

    return ret

In [143]:
access_token, expires_in = get_auth_token()

In [144]:
playlists_raw = fetch_playlists(access_token)

In [145]:
playlists = dict()
for playlist in playlists_raw['items']:
    playlists[playlist['name']] = playlist['id']

In [146]:

tracks_raw = fetch_playlist(access_token, playlists['#EDMParty'])

In [147]:
track_names = list()
for item in tracks_raw:
    for track in item['items']:
        track_names.append(track['track']['name'])

print(len(track_names))

618


In [148]:
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options, executable_path=r'E:/Program Files (x86)/Chromedriver/chromedriver.exe')
driver.get('https://music.amazon.de/search')

In [149]:
sign_in_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'signInButton'))
    )
sign_in_button.click()

# """
email_input = driver.find_element_by_name('email')
password_input = driver.find_element_by_name('password')
submit_button = driver.find_element_by_id('signInSubmit')
# """
"""
email_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, 'email'))
    )
password_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, 'password'))
    )
submit_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, 'signInSubmit'))
    )
    """
email_input.send_keys('simon.westfechtel@gmail.com')
password_input.send_keys('ho9nktdoiPme5SspSQLd')
submit_button.click()

## Initial round

In [151]:
failed_songs = []
i = 1
# track_names = track_names[:10]
print(f"tring to add {len(track_names)} songs...")

for song in track_names:
    try:
        
        time.sleep(1)
        """
        navbar_search_input = driver.find_element_by_id('navbarSearchInput')
        """
        # """
        navbar_search_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'navbarSearchInput'))
        )
        # """
        navbar_search_input.send_keys(song)
        navbar_search_input.send_keys(Keys.RETURN)

        time.sleep(1)
        
        music_container = driver.find_element_by_tag_name('music-container')
        music_container = driver.find_elements_by_tag_name('music-container')[2]
        music_item = music_container.find_element_by_tag_name('music-horizontal-item')
        builder = ActionChains(driver)
        shadow_root = driver.execute_script('return arguments[0].shadowRoot', music_item)
        div = shadow_root.find_element_by_tag_name('div')
        builder.move_to_element(div).perform()
        context_button = music_item.find_elements_by_tag_name('music-button')[1]
        shadow_root = driver.execute_script('return arguments[0].shadowRoot', context_button)
        button = shadow_root.find_element_by_tag_name('button')
        button.click()
        time.sleep(0.5)
        add_to_playlist = driver.find_element_by_id('contextMenuOption1')
        add_to_playlist.click()
        time.sleep(0.5)
        playlist = driver.find_element_by_class_name('col1')
        playlist.click()
        
        driver.get('https://music.amazon.de/search')
        if i % 100 == 0:
            print(f"elapsed {i} songs so far")
        
        i += 1
    except:
        print(f"failed to add song {song}")
        driver.get('https://music.amazon.de/search')
        failed_songs.append(song)
        continue
        
print(f"added {len(track_names) - len(failed_songs)} songs (out of {len(track_names)})")
# driver.quit()

tring to add 618 songs...
failed to add song Solace Album Mix
failed to add song Adrenalized - Original Radio Mix
failed to add song Dancing in London - David May Original Mix
failed to add song Angels (feat. Kat Nestel) - Radio Edit
failed to add song The Nights
failed to add song Get Up (Rattle) - Radio Mix
failed to add song Drvgs (feat. Hayley Gene)
failed to add song Find Tomorrow (Ocarina) (feat. Wolfpack & Katy B) - Radio Edit
failed to add song Alive
failed to add song L'Amour Toujours - Tiësto Edit
failed to add song Animal (feat. U-Jean) - Video Edit
failed to add song +1 - Radio Edit
failed to add song Higher
failed to add song Fall Out (Airbeat One Anthem 2015)
failed to add song Catch Me
failed to add song Jefrome - Unity vs Afrojack & Steve Aoki - No Beef (Jefrome Mashup)
failed to add song Bazaar (Official Sunburn Goa 2015 Anthem)
failed to add song The Nameless
failed to add song Ghosts - Courtland & EKG Remix
failed to add song Playades - Radio Eidt
failed to add song 

In [142]:
driver.quit()

## Tests

In [26]:
hydrated_elements = driver.find_elements_by_class_name('hydrated')

In [38]:
play = hydrated_elements[26]
play.click()

In [17]:
context_menu = hydrated_elements[21]
context_menu.click()

In [39]:
add_to_playlist = driver.find_element_by_id('contextMenuOption1')
add_to_playlist.click()

In [40]:
playlist = driver.find_element_by_class_name('col1')
playlist.click()

In [14]:
len(failed_songs)

604

In [130]:
music_container = driver.find_element_by_tag_name('music-container')

In [131]:
music_container = driver.find_elements_by_tag_name('music-container')[2]

In [132]:
music_item = music_container.find_element_by_tag_name('music-horizontal-item')

In [111]:
music_item = driver.find_element_by_tag_name('music-horizontal-item')

In [133]:
builder = ActionChains(driver)

In [134]:
shadow_root = driver.execute_script('return arguments[0].shadowRoot', music_item)

In [135]:
div = shadow_root.find_element_by_tag_name('div')

In [136]:
builder.move_to_element(div).perform()

In [137]:
context_button = music_item.find_elements_by_tag_name('music-button')[1]

In [138]:
shadow_root = driver.execute_script('return arguments[0].shadowRoot', context_button)

In [139]:
button = shadow_root.find_element_by_tag_name('button')

In [140]:
button.click()

In [121]:
add_to_playlist = driver.find_element_by_id('contextMenuOption1')

In [122]:
add_to_playlist.click()

In [123]:
playlist = driver.find_element_by_class_name('col1')

In [124]:
playlist.click()

In [150]:
200 % 100

0

In [152]:
len(failed_songs)

185

In [153]:
import pickle

In [154]:
with open('outfile', 'wb') as fp:
    pickle.dump(failed_songs, fp)

In [ ]:
with open ('outfile', 'rb') as fp:
    itemlist = pickle.load(fp)